### Basic Stuff
* Installing libraries
* Importing libraries
* Defining functions

In [34]:
!pip install XlsxWriter

In [13]:
# Installing libraries and all
import openpyxl
import xlsxwriter
import pathlib


import pandas as pd
import numpy as np
import json

import os.path
import base64
import io
from io import BytesIO
from urllib.request import urlopen
import re
import glob
from PIL import Image, ImageDraw, ImageOps

ALL_DATE_NAMES = ["23.2.6", "22.12.7", "23.2.28", "23.4.9", "23.1.6", "23.5.5"]
# ALL_DATE_NAMES = ['23.2.6']

def add_information_for_specific_table(table_name, table_address, date_name, table_information_dict, save_file_address, worksheet_name):
  all_images = []
  all_image_names = []
  coco_file_found = False

  table_folder_address = os.path.join(table_address, "*")
  specific_table_content = list(glob.iglob(table_folder_address))
  for item_address in specific_table_content:
      potential_image = re.findall(".*jpg|png|JPG|PNG", item_address)
      if len(potential_image)>0:
          image_name = item_address.split("/")[-1]
          all_image_names.append(image_name)

          # download image
          image = Image.open(item_address)
          image = ImageOps.exif_transpose(image)
          all_images.append(image)

      elif (item_address.split("/")[-1].lower() == 'output'):
          coco_file_found, coco_file = search_for_coco(item_address)

          if coco_file_found:
              print("COCO file found")
              bool_cat = check_categories(coco_file)
              if not bool_cat:
                  print("Didn't find correct categories in COCO file")
          else:
              print("COCO file not found")

  # sorting image names and images
  sorting_order = np.argsort(all_image_names)
  all_image_names = np.array(all_image_names)[sorting_order]
  all_images = sort_images(all_images, sorting_order)


  main_dict = initialise_main_dict()

  # going into each image
  try:
      for idx, current_image_name in enumerate(all_image_names):
          current_image_id = get_id(coco_file, current_image_name)
          current_bboxes, current_segs, current_classes, current_areas = get_coco_annotations_for_image_id(coco_file, current_image_id)

          if not ("ref" in [i.lower() for i in current_classes]):
              print(f"No ref label found")
              break

          current_bboxes = change_bboxes(current_bboxes)
          without_mask_images, with_mask_images, areas = get_final_images_and_area(all_images[idx],
                                                                                    current_bboxes,
                                                                                    current_segs,
                                                                                    current_classes,
                                                                                    current_areas)
          resize_images(with_mask_images)
          resize_images(without_mask_images)
          for final_idx, _ in enumerate(current_classes):
              main_dict['date'].append(date_name)
              main_dict['table'].append(table_name)
              main_dict['tile'].append(idx+1)
              main_dict['name'].append(current_classes[final_idx])
              main_dict['without_mask'].append(without_mask_images[final_idx])
              main_dict['with_mask'].append(with_mask_images[final_idx])
              main_dict['area'].append(areas[final_idx])

      table_number = int(re.findall("\d+", table_name)[0])

      last_row = table_information_dict[table_number]["last_row"]
      last_row = main_dict_to_excel_openpyxl(main_dict,
                                             last_row,
                                             save_file_address,
                                             worksheet_name)
      table_information_dict[table_number]["last_row"] = last_row

  except Exception as e:
        print("\nERROR:", e, "\n")
  return None

def read_coco_file(coco_file_address):
    open_file = open(coco_file_address)
    coco_file = json.load(open_file)
    return coco_file

# def search_for_coco_in_annotation_output(item_address):
#   coco_file_found = False
#   coco_file = None

#   output_content = list(glob.iglob(file_address+"/*"))
#   if len(output_content==0):
#       return coco_file_found, coco_file

#   for item_address in output_content:
#       if (item_address.split("/")[-1].lower() == 'output'):
#           coco_file_found, coco_file = search_for_coco(item_address)

#   return coco_file_found, coco_file

def search_for_coco(file_address):
    output_content = list(glob.iglob(file_address+"/*"))
    annotation_output = [i for i in output_content if i.split("/")[-1].lower() == 'annotation output']
    if len(annotation_output)>0:
        output_content = list(glob.iglob(os.path.join(annotation_output[0], "*")))
    try:
        coco_file = [i for i in output_content if "coco" in i.split("/")[-1].lower()]
        coco_file_found = True
        coco_file = read_coco_file(coco_file[0])
    except:
        coco_file_found = False
        coco_file = None
    return coco_file_found, coco_file

def initialise_main_dict():
    main_dict = {
      'date':[],
      'table':[],
      'tile':[],
      'name':[],
      'without_mask':[],
      'with_mask':[],
      'area':[]
    }
    return main_dict

def write_initial_header(name_of_file, worksheet_name):
    file_path = f"{name_of_file}.xlsx"
    if os.path.exists(file_path):
        wb = openpyxl.load_workbook(file_path)
        wb.create_sheet(worksheet_name)
        ws = wb[worksheet_name]

        ws.cell(row=1 , column=1).value="Date"
        ws.cell(row=1 , column=2).value="Table"
        ws.cell(row=1 , column=3).value="Tile"
        ws.cell(row=1 , column=4).value="Name"
        ws.cell(row=1 , column=5).value="Original"
        ws.cell(row=1 , column=6).value="With Mask"
        ws.cell(row=1 , column=7).value="Area"

        wb.save(file_path)

        wb.close()
    else:
        # use existing xlsx code
        workbook = xlsxwriter.Workbook(file_path)
        worksheet = workbook.add_worksheet(worksheet_name)

        worksheet.write(0, 0, "Date")
        worksheet.write(0, 1, "Table")
        worksheet.write(0, 2, "Tile")
        worksheet.write(0, 3, "Name")
        worksheet.write(0, 4, "Original")
        worksheet.write(0, 5, "With Mask")
        worksheet.write(0, 6, "Area")

        workbook.close()


def main_dict_to_excel_openpyxl(main_dict, last_row, name_of_file, worksheet_name):
    file_path = f'{name_of_file}.xlsx'
    wb = openpyxl.load_workbook(file_path)
    ws = wb[worksheet_name]

    number_entries = len(main_dict["date"])

    for row_number in range(2, number_entries+2):
        for col_number, title in enumerate(main_dict.keys(), start=1):
            value = main_dict[title][row_number-2]
            if title in ["date", 'table', 'tile', 'name', 'area']:
                ws.cell(row=row_number+last_row, column=col_number).value = value
            else:
                img_byte_arr = io.BytesIO()
                value.save(img_byte_arr, format='png')
                img = Image.open(img_byte_arr)
                img = openpyxl.drawing.image.Image(img)
                img.height = 20
                img.width = 64
                img.anchor = chr(64+col_number) + str(row_number+last_row)
                ws.add_image(img)
    wb.save(file_path)
    last_row += number_entries
    return last_row


def main_dict_to_excel_xls(main_dict):
    workbook = xlsxwriter.Workbook("Analysis-Excel.xlsx")
    worksheet = workbook.add_worksheet()
    number_entries = len(main_dict["date"])

    # image-related definitions
    cell_width = 64.0
    cell_height = 20.0

    image_width, image_height = main_dict["without_mask"][0].size

    x_scale = cell_width/image_width
    y_scale = cell_height/image_height



    for row_number in range(1, number_entries+1):
        for col_number, title in enumerate(main_dict.keys()):
            value = main_dict[title][row_number-1]
            if title=="date":
                worksheet.write(row_number, col_number, value)
            elif title=="table":
                worksheet.write(row_number, col_number, value)
            elif title=="tile":
                worksheet.write(row_number, col_number, value)
            elif title=="name":
                worksheet.write(row_number, col_number, value)
            elif title=="without_mask":
                img_byte_arr = io.BytesIO()
                value.save(img_byte_arr, format='PNG')
                image_data = img_byte_arr
                worksheet.insert_image(row_number, col_number, 'random.png', {"image_data": image_data, 'x_scale': x_scale, 'y_scale': y_scale})
            elif title=="with_mask":
                img_byte_arr = io.BytesIO()
                value.save(img_byte_arr, format='PNG')
                image_data = img_byte_arr
                worksheet.insert_image(row_number, col_number, 'random.png', {"image_data": image_data, 'x_scale': x_scale, 'y_scale': y_scale})
            elif title=="area":
                worksheet.write(row_number, col_number, value)
    workbook.close()

def query_search(query_id):
    files = []
    page_token = None
    while True:
        response = service.files().list(q=f"'{query_id}' in parents",
                                        spaces='drive',
                                        fields='files(id, name)').execute()
        for file in response.get('files', []):
            # Process change
            file["name"] = file['name'].lower().strip()
        files.extend(response.get('files', []))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    return files

def download_gdrive_file(fileId):
    request = service.files().get_media(fileId=fileId)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      print(f"Download {int(status.progress() * 100)}.")
    return file

def check_categories(coco_file):
    cats = coco_file["categories"]
    cat_names = [i['name'].lower() for i in cats]
    if 'coral_a' in cat_names:
        return True
    else:
        return False

def get_id(coco_file, current_image_name):
    current_image_id = [i for i in coco_file['images'] if i['file_name'] == current_image_name][0]["id"]
    return current_image_id

def get_coco_annotations_for_image_id(coco_file, current_image_id):
    annotations = [i for i in coco_file['annotations'] if i['image_id'] == current_image_id]
    segmentations = [i['segmentation'] for i in annotations]
    category_ids = [i["category_id"] for i in annotations]
    classes = [[i["name"] for i in coco_file['categories'] if i["id"] == j][0] for j in category_ids]
    bboxes = create_bboxes_from_segs(segmentations)
#     bboxes = [i['bbox'] for i in annotations]
    areas = [i["area"] for i in annotations]
    return bboxes, segmentations, classes, areas

def create_bboxes_from_segs(segs):
    bboxes = []
    for seg in segs:
        xy = [(seg[0][idx*2], seg[0][idx*2+1]) for idx, i in enumerate(seg[0][0:-1:2])]
        x0 = 10000000
        y0 = 10000000
        x1 = -1
        y1 = -1
        for point in xy:
            if point[0]>x1:
                x1 = point[0]
            if point[0]<x0:
                x0 = point[0]
            if point[1]>y1:
                y1 = point[1]
            if point[1]<y0:
                y0 = point[1]
        bbox = [x0, y0, x1-x0, y1-y0]
        bboxes.append(bbox)
    return bboxes

def change_bboxes(bboxes):
    new_bboxes = []
    for bbox in bboxes:
        bbox[0] -= 100
        bbox[1] -= 100
        bbox[2] += 200
        bbox[3] += 200
        new_bboxes.append(bbox)
    return new_bboxes

def get_final_images_and_area(image, bboxes, segmentations, classes, areas):
    # get image with masks
    image_with_masks = layer_image_with_mask(image, segmentations)

    # get cropped images
    bbox_without_mask_images = []
    bbox_with_mask_images = []
    for bbox in bboxes:
        cropped_image = crop_image(image, bbox)
        bbox_without_mask_images.append(cropped_image)

        cropped_image = crop_image(image_with_masks, bbox)
        bbox_with_mask_images.append(cropped_image)

    # get areas of all the corals
    cm_2_areas = calculate_areas(classes, areas)

    return bbox_without_mask_images, bbox_with_mask_images, cm_2_areas

def crop_image(image, bbox):
    x0, y0, w, h = bbox
    cropped_image = image.crop((x0, y0, x0+w, y0+h))
    return cropped_image

def layer_image_with_mask(image, segmentations):
    image = image.convert('RGBA')
    image_copy = image.copy()
    draw = ImageDraw.Draw(image_copy)

    for seg in segmentations:
        xy = [(seg[0][idx*2], seg[0][idx*2+1]) for idx, i in enumerate(seg[0][0:-1:2])]
        draw.polygon(xy, fill = (255, 255, 0))
    layered_image = Image.blend(image, image_copy, 0.5)
    return layered_image


def calculate_areas(classes, areas):
    ref_idx = [idx for idx, i in enumerate(classes) if i.lower()=='ref'][0]
    ref_area = areas[ref_idx]
    cm_2_areas = [i * 25/ref_area for i in areas]
    return cm_2_areas

def sort_images(list_of_images, order):
    array_images = [np.asarray(i) for i in list_of_images]
    # array_images = np.array(array_images, dtype=object)[order]
    # return [Image.fromarray(i.astype(np.uint8)) for i in array_images]
    sorted_arrays = [x for _, x in sorted(zip(order, array_images), key=lambda pair: pair[0])]
    return [Image.fromarray(i) for i in sorted_arrays]

def resize_images(images_list):
    size = (256, 256)
    for i in images_list:
        i.thumbnail(size)
    return None

### Defining Custom Variables

In [24]:
main_dir_address = "/content/drive/MyDrive/Projects/Coral Microfragmentation/Coral Monitoring/22-23 Season/" #@param {type:"string"}
date_name = "" #@param {type:"string"}
intermediary_folder_path = "" #@param {type:"string"}
table_number = 10 #@param {type:"integer"}

### Fixed Variables Being Defined and Running the Main Function

In [25]:
if date_name:
    table_names = ["table_" + str(table_number)]
    table_folder_addresses = [os.path.join(main_dir_address, date_name, intermediary_folder_path, table_names[0])]
    save_file_name = "Analysis-Excel"
    save_file_name_extension = save_file_name + '.xlsx'
    save_file_path = os.path.join(table_folder_addresses[0], "analysis_file", save_file_name)
    worksheet_names = ["Main Analysis"]
else:
    print('Create files across all dates for given table...')
    table_names = ["table_" + str(table_number)] * len(ALL_DATE_NAMES)
    table_folder_addresses = []
    worksheet_names = []
    for date_name in ALL_DATE_NAMES:
        table_folder_addresses.append(
            os.path.join(main_dir_address, date_name, intermediary_folder_path, table_names[0])
        )
        worksheet_names.append(f"date_{date_name}")
    save_file_name = f"Analysis-Excel_table_{str(table_number)}"
    save_file_name_extension = save_file_name + '.xlsx'
    save_file_path = os.path.join(main_dir_address, "analysis_files", save_file_name)


for idx, table_name in enumerate(table_names):
    print(f'Progress: {(idx+1)/len(table_names)}')

    table_folder_address = table_folder_addresses[idx]
    worksheet_name = worksheet_names[idx]

    # initialising some needed files and dictionary
    pathlib.Path("/".join(save_file_path.split("/")[:-1])).mkdir(parents=True, exist_ok=True)
    table_information_dict = dict()
    table_information_dict[table_number] = {"last_row":0}
    write_initial_header(save_file_path, worksheet_name)
    add_information_for_specific_table(table_name, table_folder_address, date_name, table_information_dict, save_file_path, worksheet_name)

Create files across all dates for given table...
Progress: 0.16666666666666666
COCO file found
Progress: 0.3333333333333333
COCO file found
Progress: 0.5
COCO file found
Progress: 0.6666666666666666
COCO file found
Progress: 0.8333333333333334
COCO file found
Progress: 1.0
COCO file found


### Rough

In [ ]:
# initialising some need files and dictionary

table_information_dict = dict()
for i in range(starting_table_number, ending_table_number+1):
    table_information_dict[i] = {"last_row":0}
    write_initial_header("Analysis-Excel"+str(i))

In [ ]:
# defining custom variables
dates_to_check = ["22.12.7", "23.1.6", "23.2.6", "23.2.28", "23.4.9", "23.5.5"]
tables_to_check = ["table_4", "table_5", "table_6", "table_7", "table_8", "table_9", "table_10"]

dates_to_check = ["23.1.6"]
tables_to_check = ["table_4"]

In [ ]:
monitoring_content = query_search("1fdjJD2nMX9V-8ddP4FjqpzoD-KnCI71P")
for date_folder in monitoring_content:
    if date_folder['name'] in dates_to_check:

        current_date_id = date_folder['id']
        date_content = query_search(current_date_id)

        date_content_names = [i["name"] for i in date_content]
        date_content_ids = [i["id"] for i in date_content]
        try:
            idx_new_tables = date_content_names.index('new tables')
            id_new_tables = date_content_ids[idx_new_tables]
            new_table_content = query_search(id_new_tables)
        except:
            try:
                idx_new_tables = date_content_names.index('coral tables')
                id_new_tables = date_content_ids[idx_new_tables]
                new_table_content = query_search(id_new_tables)
                new_table_content = query_search([i["id"] for i in new_table_content if i['name'] == 'new tables'][0])
            except:
                try:
                    idx_new_tables = date_content_names.index('post cleaning')
                    id_new_tables = date_content_ids[idx_new_tables]
                    new_table_content = query_search(id_new_tables)
                except:
                    new_table_content = date_content
        for table_folder in new_table_content:
            if (table_folder["name"] in tables_to_check) or (table_folder["name"].replace(" ", "_") in tables_to_check):
                print(f"---Checking for: Date-{date_folder['name']}, Table-{table_folder['name']}---")
                specific_table_content = query_search(table_folder['id'])

In [ ]:
monitoring_content = query_search("1fdjJD2nMX9V-8ddP4FjqpzoD-KnCI71P")
for date_folder in monitoring_content:
    if date_folder['name'] in dates_to_check:

        current_date_id = date_folder['id']
        date_content = query_search(current_date_id)

        date_content_names = [i["name"] for i in date_content]
        date_content_ids = [i["id"] for i in date_content]
        try:
            idx_new_tables = date_content_names.index('new tables')
            id_new_tables = date_content_ids[idx_new_tables]
            new_table_content = query_search(id_new_tables)
        except:
            try:
                idx_new_tables = date_content_names.index('coral tables')
                id_new_tables = date_content_ids[idx_new_tables]
                new_table_content = query_search(id_new_tables)
                new_table_content = query_search([i["id"] for i in new_table_content if i['name'] == 'new tables'][0])
            except:
                try:
                    idx_new_tables = date_content_names.index('post cleaning')
                    id_new_tables = date_content_ids[idx_new_tables]
                    new_table_content = query_search(id_new_tables)
                except:
                    new_table_content = date_content
        for table_folder in new_table_content:
            if (table_folder["name"] in tables_to_check) or (table_folder["name"].replace(" ", "_") in tables_to_check):
                print(f"---Checking for: Date-{date_folder['name']}, Table-{table_folder['name']}---")
                specific_table_content = query_search(table_folder['id'])

                all_images = []
                all_image_names = []
                coco_file_found = False
                for item in specific_table_content:

                    potential_image = re.findall(".*jpg|png", item['name'])
                    if len(potential_image)>0:
                        image_name = potential_image[0]
                        all_image_names.append(image_name)
                        image_id = item['id']

                        #download image
                        file = download_gdrive_file(image_id)
                        image = Image.open(io.BytesIO(file.getvalue()))
                        image = ImageOps.exif_transpose(image)
                        all_images.append(image)

                    elif item['name'] == 'output':
                        output_content = query_search(item["id"])
                        annotation_id = [i["id"] for i in output_content if i["name"]=="annotation output"][0]
                        annotation_content = query_search(annotation_id)
                        try:
                            coco_file_id = [i["id"] for i in annotation_content if "coco" in i["name"]][0]
                        except:
                            print(f"coco file not found for {date_folder['name']}, {table_folder['name']}")
                            break


                        # download coco file
                        file = download_gdrive_file(coco_file_id)
                        coco_file = literal_eval(file.getvalue().decode("utf-8"))
                        bool_cat = check_categories(coco_file)
                        coco_file_found = True
                        if not bool_cat:
                            print(("Didn't find correct categories in COCO file of "
                                   f"{date_folder['name']}-{table_folder['name']}"))

                if not coco_file_found:
                    print(f"coco file not found for {date_folder['name']}, {table_folder['name']}")
                    continue

                sorting_order = np.argsort(all_image_names)
                all_image_names = np.array(all_image_names)[sorting_order]
                all_images = sort_images(all_images, sorting_order)


                main_dict = {
                    'date':[],
                    'table':[],
                    'tile':[],
                    'name':[],
                    'without_mask':[],
                    'with_mask':[],
                    'area':[]
                }
                # going into each image
                print("Looping through all the image names now...")
                try:
                    for idx, current_image_name in enumerate(all_image_names):
                        current_image_id = get_id(coco_file, current_image_name)
                        current_bboxes, current_segs, current_classes, current_areas = get_coco_annotations_for_image_id(coco_file, current_image_id)

                        if not ("ref" in [i.lower() for i in current_classes]):
                            print(f"No ref label found for {date_folder['name']}, {table_folder['name']}")
                            break

                        current_bboxes = change_bboxes(current_bboxes)
                        without_mask_images, with_mask_images, areas = get_final_images_and_area(all_images[idx],
                                                                                                 current_bboxes,
                                                                                                 current_segs,
                                                                                                 current_classes,
                                                                                                 current_areas)
                        resize_images(with_mask_images)
                        resize_images(without_mask_images)
                        for final_idx, _ in enumerate(current_classes):
                            main_dict['date'].append(date_folder['name'])
                            main_dict['table'].append(table_folder['name'])
                            main_dict['tile'].append(idx+1)
                            main_dict['name'].append(current_classes[final_idx])
                            main_dict['without_mask'].append(without_mask_images[final_idx])
                            main_dict['with_mask'].append(with_mask_images[final_idx])
                            main_dict['area'].append(areas[final_idx])

                    table_number = int(re.findall("\d+", table_folder["name"])[0])

                    last_row = table_information_dict[table_number]["last_row"]
#                     last_row = main_dict_to_excel_openpyxl(main_dict,
#                                                            last_row,
#                                                            "Jeremy_Analysis_"+str(table_number))
                    table_information_dict[table_number]["last_row"] = last_row

                except Exception as e:
                      print("\nERROR:", e, "\n")